# Pandas and SQL

* Select rows
* Merge and Join
* Filter and Where
* Sort and Order
* Group
* decile
* window function qualify rank

In [3]:
import seaborn as sns
import numpy as np
import pandas as pd

df_demo = sns.load_dataset('titanic')

## Select

* select * from

In [4]:
# SQL 
# select * from table limit 10

df_demo.head(10)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False


In [5]:
# SQL
# select sex, age from df_demo

df_demo[['sex','age']]

,sex,age
0,male,22.0
1,female,38.0
2,female,26.0
3,female,35.0
4,male,35.0
...,...,...
886,male,27.0
887,female,19.0
888,female,NaN
889,male,26.0


In [6]:
# SQL
# select distinct sex from df_demo
df_demo['sex'].drop_duplicates()

0      male
1    female
Name: sex, dtype: object

In [7]:
# SQL
# select sex as gender, age as age_new from df_demo
(
df_demo[['sex','age']].
 rename(columns={'sex':'gender',
                'age':'age_new'})
)

,gender,age_new
0,male,22.0
1,female,38.0
2,female,26.0
3,female,35.0
4,male,35.0
...,...,...
886,male,27.0
887,female,19.0
888,female,NaN
889,male,26.0


In [12]:
# SQL
# select case when age < 60 then '<60' 
# when age >= 60 then '>=60'
# else 'Unknown'
# end as age_band 
# from df_demo

conditions = [df_demo['age']<60
              ,df_demo['age']>=60]
label= ['<60', '>=60']

df_demo['age_band'] = np.select(conditions,label,default='Unknown')
df_demo.age_band.value_counts()

<60        688
Unknown    177
>=60        26
Name: age_band, dtype: int64

In [14]:
df_demo['age_band'] = np.select(
    [
        df_demo['age'].between(0, 60, inclusive=False), 
        df_demo['age'].between(60, 100, inclusive=True)
    ], 
    [
        'age <60', 
        'age >=60'
    ], 
    default='Unknown'
)
df_demo.age_band.value_counts()

age <60     688
Unknown     177
age >=60     26
Name: age_band, dtype: int64

## Merge and Join

In [15]:
# select from df_demo a left join df_demo b on a.id=b.id

# df.merge(df2, left_on='id',right_on='id',how='left')


In [16]:
# select * from a union all select * from b
# df = pd.concat([a,b])


## Filter

In [17]:
# select * from df_demo where age=60

df_demo[df_demo['age']==60]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_band
366,1,1,female,60.0,1,0,75.25,C,First,woman,False,D,Cherbourg,yes,False,age >=60
587,1,1,male,60.0,1,1,79.20,C,First,man,True,B,Cherbourg,yes,False,age >=60
684,0,2,male,60.0,1,1,39.00,S,Second,man,True,NaN,Southampton,no,False,age >=60
694,0,1,male,60.0,0,0,26.55,S,First,man,True,NaN,Southampton,no,True,age >=60


In [19]:
# select * from df where age=60 and sex=male

df_demo[(df_demo['age']==60)&(df_demo['sex']=='male')]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_band
587,1,1,male,60.0,1,1,79.20,C,First,man,True,B,Cherbourg,yes,False,age >=60
684,0,2,male,60.0,1,1,39.00,S,Second,man,True,NaN,Southampton,no,False,age >=60
694,0,1,male,60.0,0,0,26.55,S,First,man,True,NaN,Southampton,no,True,age >=60


## Order by

In [20]:
# select * from df order by age desc
df_demo.sort_values('age',ascending=False)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_band
630,1,1,male,80.0,0,0,30.0000,S,First,man,True,A,Southampton,yes,True,age >=60
851,0,3,male,74.0,0,0,7.7750,S,Third,man,True,NaN,Southampton,no,True,age >=60
493,0,1,male,71.0,0,0,49.5042,C,First,man,True,NaN,Cherbourg,no,True,age >=60
96,0,1,male,71.0,0,0,34.6542,C,First,man,True,A,Cherbourg,no,True,age >=60
116,0,3,male,70.5,0,0,7.7500,Q,Third,man,True,NaN,Queenstown,no,True,age >=60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,0,3,male,NaN,0,0,7.2292,C,Third,man,True,NaN,Cherbourg,no,True,Unknown
863,0,3,female,NaN,8,2,69.5500,S,Third,woman,False,NaN,Southampton,no,False,Unknown
868,0,3,male,NaN,0,0,9.5000,S,Third,man,True,NaN,Southampton,no,True,Unknown
878,0,3,male,NaN,0,0,7.8958,S,Third,man,True,NaN,Southampton,no,True,Unknown


## Group by

In [21]:
# select sex,count(distinct age) from df group by sex

df_demo.groupby('sex')['age'].nunique()

sex
female    63
male      82
Name: age, dtype: int64

In [22]:
# select survived, avg(fare) from df group by survived

df_demo.groupby('survived')['fare'].mean()

survived
0    22.117887
1    48.395408
Name: fare, dtype: float64